<a href="https://colab.research.google.com/github/scativa/IV-Colab/blob/main/segmentation_pretrained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1 - Librerias

##functions.py
Reemplaza la línea `import functions as func`

In [ ]:
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


'''
Some times I need normalizad images
'''
def load_images(dir, filenames=None, norm=False):
  if filenames is None:
    print(f"Cargando {len(os.listdir(dir))} imagenes... ")
    filenames = os.listdir(dir) # Cuidado, esto no garantiza el orden.
  else:
    print(f"load_images {len(filenames)}... ")

  images = []
  i = 0
  for filename in filenames:
    # print(f"{filename}: {len(images)}/{len(filenames)}",end="\r")
    print(f"{i}) {filename}") # ,end="\r")
    i+=1
    image = cv2.imread(os.path.join(dir, filename))
    if image is not None:
      images.append(image.astype(np.float32))
  # images = [cv2.imread(os.path.join(dir, filename)).astype(np.float32)
  #         for filename in filenames
  #         if cv2.imread(os.path.join(dir, filename)) is not None]
  if norm:
      images = [image / 255.0 for image in images]

  # https://chatgpt.com/c/67fd5724-8164-8006-9367-780189493ee5
  images = [cv2.resize(image, (224, 224)).astype(np.float32) for image in images]

  return images


def load_masks(dir_pores, image_filenames):

  for filename in image_filenames:
    if not os.path.exists(os.path.join(dir_pores, filename)):
      print(f"Warning: {os.path.join(dir_pores, filename)} does not exist")

  masks = [
          cv2.imread(os.path.join(dir_pores, filename), cv2.IMREAD_GRAYSCALE).astype(np.float32)
          # if os.path.exists(os.path.join(dir_pores, filename))
          # else np.zeros((232, 180), dtype=np.float32)
          for filename in image_filenames
  ]
  # https://chatgpt.com/c/67fd5724-8164-8006-9367-780189493ee5
  masks = [cv2.resize(mask, (224, 224)).astype(np.float32) for mask in masks]
  masks = [np.expand_dims(mask, axis=-1) for mask in masks] #adds a channel dimension to the masks


  return masks

# just to plot masks
def plot_img_mask(img, mask, pred):

    fig, axes = plt.subplots(3, 1, figsize=(5, 15))

    sub_plot = 0

    if (img is not None):
      axes[sub_plot].imshow(img)
      axes[sub_plot].set_title('Image')
      axes[sub_plot].axis('off')
      sub_plot += 1

    if (mask is not None):
      axes[sub_plot].imshow(mask, cmap='gray')
      axes[sub_plot].set_title('Mask')
      axes[sub_plot].axis('off')
      sub_plot += 1

    if (pred is not None):
      pred_mask = np.where(pred <= 0.5, 0, 1)

      axes[sub_plot].imshow(pred_mask, cmap='gray')
      axes[sub_plot].set_title('Predicted')
      axes[sub_plot].axis('off')

    plt.tight_layout()
    plt.show()

def augmented_dataset(dir_images, dir_mask, seed, norm):
    # From "Pores 03/28/25 -Seba- Basado en segmentation_own (DTE_fisuras).ipynb"

    # image_filenames = os.path.basename(sorted(os.listdir(dir_images)))
    image_filenames = sorted(os.listdir(dir_images))

    # https://chatgpt.com/c/681d085f-e2ec-8006-833f-939f942ffd5b
    # Mejoras para la carga de imágenes
    images = load_images(dir_images, image_filenames, norm)
    # Get image filenames from loaded images
    # image_filenames = [os.path.basename(image_path) for image_path in os.listdir(dir_images)]
    # masks = load_masks(dir_pores)

    masks_filenames = [f"{(p:=img_name.split('-'))[0]}-{p[1]}-{p[4]}-avg{p[6][-7:]}" for img_name in image_filenames]
    print(image_filenames[0], masks_filenames[0])
    masks = load_masks(dir_mask, masks_filenames)
    # masks = load_masks(dir_mask, image_filenames)


    X_train, X_test, y_train, y_test, filenames_train, filenames_test = train_test_split(images, masks, image_filenames, test_size=0.2, random_state=seed)

    X_train[0].shape
    y_train[0].shape
    augmented_X_train = [cv2.flip(i, 1) for i in X_train]
    augmented_y_train = [cv2.flip(i, 1) for i in y_train]
    augmented_y_train = [np.expand_dims(mask, axis=-1) for mask in augmented_y_train ] #adds a channel dimension to the masks

    # print(y_train[0].shape, augmented_y_train[0].shape)
    X_train = np.array(X_train + augmented_X_train)
    y_train = np.array(y_train + augmented_y_train)

    X_test = np.array(X_test)
    y_test = np.array(y_test)


    # y_train = np.expand_dims(y_train, axis=-1)
    # y_val = np.expand_dims(y_val, axis=-1)
    # y_test = np.expand_dims(y_test, axis=-1)

    return X_train, y_train, X_val, y_val, X_test, y_test, filenames_train, filenames_val, filenames_test

def load_img_mask(dir_images, dir_mask, image_filenames, masks_filenames, norm):
    print(f"Cargando {len(image_filenames)} imagenes... ")
    X = load_images(dir_images, image_filenames, norm)
    # masks_filenames = [f"{(p:=img_name.split('-'))[0]}-{p[1]}-{p[4]}-avg{p[6][-7:]}" for img_name in image_filenames]
    print(f"Cargando {len(image_filenames)} máscaras... ")
    y = load_masks(dir_mask, masks_filenames)

    return X, y

def augmented_dataset_ETIl(dir_images, dir_mask, filenames_train, filenames_val, filenames_train_masks, filenames_val_masks, norm):
# def augmented_dataset_ETIl(dir_images, dir_mask, filenames_train, filenames_val, filenames_test, filenames_train_masks, filenames_val_masks, filenames_test_masks, norm):

    print(f"Cargadno Train dataset")
    X_train, y_train = load_img_mask(dir_images, dir_mask, filenames_train, filenames_train_masks, norm)
    print(f"Cargadno Val dataset")
    X_val, y_val = load_img_mask(dir_images, dir_mask, filenames_val, filenames_val_masks, norm)
    # print(f"Cargadno Test dataset")
    # X_test, y_test = load_img_mask(dir_images, dir_mask, filenames_test, filenames_test_masks, norm)

    augmented_X_train = [cv2.flip(i, 1) for i in X_train]
    augmented_y_train = [cv2.flip(i, 1) for i in y_train]
    augmented_y_train = [np.expand_dims(mask, axis=-1) for mask in augmented_y_train ] #adds a channel dimension to the masks

    # print(y_train[0].shape, augmented_y_train[0].shape)
    X_train = np.array(X_train + augmented_X_train)
    y_train = np.array(y_train + augmented_y_train)

    # X_test = np.array(X_test)
    # y_test = np.array(y_test)

    return X_train, y_train, X_val, y_val
    # return X_train, y_train, X_val, y_val, X_test, y_test, filenames_train, filenames_val, filenames_test


def test_dataset(dir_images, dir_mask, seed, norm):
  # sólo el conjunto definido para test
  image_filenames = sorted(os.listdir(dir_images))

  images = load_images(dir_images, image_filenames, norm)
  # Get image filenames from loaded images
  # image_filenames = [os.path.basename(image_path) for image_path in os.listdir(dir_images)]
  # masks = load_masks(dir_pores)
  masks = load_masks(dir_mask, image_filenames)

  _, X_test, _, y_test, _, filenames_test = train_test_split(images, masks, test_size=0.2, random_state=seed)

  # y_test = np.expand_dims(y_test, axis=-1)

  return X_test, y_test, filenames_test

'''
If you want to show some other metrics, just uncomment
'''
def plot_history(history, x_lim, y_lim):
    #val_iou_score = history.history['val_iou_score']
    #iou_score = history.history['iou_score']
    val_f1_score = history.history['val_f1-score']
    #f1_score = history.history['f1-score']
    val_loss = history.history['val_loss']
    loss = history.history['loss']
    epochs = range(1, len(loss) + 1)

    # Plotting
    plt.figure(figsize=(12, 6))

    #plt.plot(epochs, val_iou_score, 'b', label='Validation IoU Score')
    #plt.plot(epochs, iou_score, 'g', label='IoU Score')
    #plt.plot(epochs, f1_score, 'r', label='F1 Score')
    plt.plot(epochs, val_f1_score, 'c', label='Validation F1 Score')
    plt.plot(epochs, val_loss, 'm', label='Validation Loss')
    plt.plot(epochs, loss, 'y', label='Training Loss')

    plt.title('Training Metrics', fontsize=16)
    plt.xlabel('Epochs', fontsize=14)
    plt.ylabel('Score/Loss', fontsize=14)
    plt.legend(fontsize=12)

    plt.grid(True)
    plt.xlim(1, x_lim)
    plt.ylim(0, y_lim)
    plt.show()

#2 - Entorno

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
from time import process_time_ns
import os

import os, json

json_path = os.path.join('/content/drive/MyDrive/', 'drvinfo.json')
usr_drive = json.load(open(json_path)).get('username') if os.path.exists(json_path) else None

base_paths = { # Path al
    "ppca.cnea": '/content/drive/MyDrive/IV',
    "seba.cnea": '/content/drive/MyDrive/CNEA/DCA/Proyectos/IV - Inspección Visual/IV',
    "scativa": '/content/drive/MyDrive/Laburo/CNEA/DCA/Proyectos/IV - Inspección Visual/IV'
}
base_path = base_paths[usr_drive]

if not os.path.exists(base_path):
  print(f'Carpeta inexistente "{usr_drive}: {base_path}"')
else:
  print(f'Utilizando carpeta  "{usr_drive}: {base_path}"')


# import tensorflow as tf
# # Setting GPU memory growth to True
# gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#     print(f"GPUs detected: {gpus}")
#     try:
#         for gpu in gpus:
#             tf.config.experimental.set_memory_growth(gpu, True)
#     except RuntimeError as e:
#         print(e)


os.environ["SM_FRAMEWORK"] = "tf.keras"
from tensorflow import keras
from keras import backend as K


Utilizando carpeta  "scativa: /content/drive/MyDrive/Laburo/CNEA/DCA/Proyectos/IV - Inspección Visual/IV"


# 3 - Modelos

In [ ]:
!pip install segmentation-models
import segmentation_models as sm
from segmentation_models import get_preprocessing

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 3.7 MB/s eta 0:00:00
Segmentation Models: using `tf.keras` framework.


In [ ]:
import tensorflow as tf
import segmentation_models as sm

# === Registro para serialización ===
@tf.keras.utils.register_keras_serializable(package="Custom")
def custom_bce_dice_loss(y_true, y_pred):
    return sm.losses.bce_dice_loss(y_true, y_pred)

@tf.keras.utils.register_keras_serializable(package="Custom")
def custom_bce_jaccard_loss(y_true, y_pred):
    return sm.losses.bce_jaccard_loss(y_true, y_pred)

@tf.keras.utils.register_keras_serializable(package="Custom")
def custom_iou_score(y_true, y_pred):
    return sm.metrics.iou_score(y_true, y_pred)

@tf.keras.utils.register_keras_serializable(package="Custom")
def custom_f1_score(y_true, y_pred):
    return sm.metrics.f1_score(y_true, y_pred)

In [ ]:
model_resnet_152 = None
model_vgg19 = None
model_efficientnetb2 = None
preprocess_input = None

def create_model(model_type):
  global model_resnet_152, model_vgg19, model_efficientnetb2, preprocess_input

  if model_type == "resnet152":
    print("Cargando ResNet152...")
    preprocess_input_resnet152 = get_preprocessing('resnet152')
    preprocess_input = preprocess_input_resnet152
    model_resnet_152 = sm.Unet(model_type, classes=1, activation='sigmoid', encoder_weights='imagenet')
    model_resnet_152.compile(
        'Adam',
        loss=custom_bce_jaccard_loss, # Use the custom registered loss
        metrics=[custom_iou_score, custom_f1_score],
    )

  elif model_type == "vgg19":
    print("Cargando VGG19...")
    preprocess_input_vgg19 = get_preprocessing('vgg19')
    model_vgg19 = sm.Unet(model_type, classes=1, activation='sigmoid', encoder_weights='imagenet')
    model_vgg19.compile(
        'Adam',
        loss=custom_bce_jaccard_loss, # Use the custom registered loss
        metrics=[custom_iou_score, custom_f1_score],
    )

  elif model_type == "efficientnetb2":
    print("Cargando EfficientNetB2...")
    preprocess_input_efficientnetb2 = get_preprocessing('efficientnetb2')
    model_efficientnetb2 = sm.Unet(model_type, classes=1, activation='sigmoid', encoder_weights='imagenet')
    model_efficientnetb2.compile(
        'Adam',
        loss=custom_bce_jaccard_loss, # Use the custom registered loss
        metrics=[custom_iou_score, custom_f1_score],
  )

# 4 - Dataset

In [ ]:
defect = 'pores' #'cachaduras'

image_directory = f'{base_path}/Images/Output_imágenes y máscaras 20250318/250127_p_2_i11'
masks_directory = f'{base_path}/Images/Output_imágenes y máscaras 20250318/all_{defect}_250520'
# masks_directory = f'{base_path}/Images/Output_imágenes y máscaras 20250318/all_cachaduras_250520'
# masks_directory = f'{base_path}/Images/Output_imágenes y máscaras 20250318/all_cachaduras'

if not os.path.exists(image_directory):
  print(f'Carpeta inexistente "{usr_drive}: {base_path}"')
else:
  print(f'Utilizando carpeta  "{usr_drive}: {base_path}"')

if not os.path.exists(masks_directory):
  print(f'Carpeta inexistente "{usr_drive}: {base_path}"')
else:
  print(f'Utilizando carpeta  "{usr_drive}: {base_path}"')

# print(f"{image_directory}: ",os.path.exists(image_directory))
# print(f"{masks_directory}: ",os.path.exists(masks_directory))



Utilizando carpeta  "scativa: /content/drive/MyDrive/Laburo/CNEA/DCA/Proyectos/IV - Inspección Visual/IV"
Utilizando carpeta  "scativa: /content/drive/MyDrive/Laburo/CNEA/DCA/Proyectos/IV - Inspección Visual/IV"


In [ ]:
import cv2
import numpy as np
import os
import tensorflow as tf
import random

# https://chatgpt.com/c/6838bd09-1c94-8010-aae4-2ec5f990ecb7

class SegmentationDataGenerator(tf.keras.utils.Sequence):
    # def __init__(self, img_dir, mask_dir, img_filenames, mask_filenames,
    #              batch_size=8, preprocess_fn=None, norm=True,
    #              shuffle=True, augment=False):
    def __init__(self, img_dir, mask_dir, samples, fn_data,
                 batch_size=8, preprocess_fn=None, norm=True,
                 shuffle=True, augment=False):
        self.img_dir = img_dir
        self.mask_dir = mask_dir

        self.samples = samples
        # self.img_filenames = img_filenames
        # self.mask_filenames = mask_filenames

        self.batch_size = batch_size
        self.preprocess_fn = preprocess_fn
        self.norm = norm
        self.shuffle = shuffle
        self.augment = augment
        self.indices = np.arange(len(self.samples))
        # self.indices = np.arange(len(self.img_filenames))
        self.on_epoch_end()

        self.fn_data = fn_data
        #  {
        #   "fecha": "250127",
        #   "series": ["BA"],
        #   # "rotaciones": [
        #   #   "1", "2", "3", "4", "5", "6", "7", "8", "9", "10",
        #   #   "11", "12", "13", "14", "15", "16", "17", "18", "19", "20",
        #   #   "21", "22", "23", "24", "25", "26", "27", "28", "29", "30",
        #   #   "31", "32", "33", "34", "35", "36", "37", "38", "39", "40",
        #   #   "41", "42", "43", "44", "45", "46", "47", "48", "49", "50"
        #   # ],
        #   "iluminaciones": [
        #     "0-11000000", "0-00110000", "0-00001100", "0-00000011", "1-11000000"],
        #   "ET": [
        #     "30000.00", "32000.00", "34000.00", "36000.00", "38000.00"],
        #   # "hileras": ["A", "B", "C", "D", "E", "F"],
        #   # "filas": ["0", "1", "2", "3", "4"],
        #   "ext": "png",

        #   "fn_img_patt": "{fecha}-r{rotacion}-i{iluminacion}-{serie}-ET{et}-a0000_{pastilla}.{ext}",
        #   "fn_msk_patt": "{fecha}-r{rotacion}-{serie}-avg_{pastilla}.{ext}"

        # }


    def __len__(self):
        return int(np.ceil(len(self.samples) / self.batch_size))
        # return int(np.ceil(len(self.img_filenames) / self.batch_size))

    def __getitem__(self, index):
        idxs = self.indices[index * self.batch_size : (index + 1) * self.batch_size]
        batch_imgs = []
        batch_masks = []




        for i in idxs:
            r, p = self.samples[i].split("-")
            r = r[1:]

            img_filename = self.fn_data["fn_img_patt"].format(
                fecha=self.fn_data["fecha"],
                rotacion=r,
                iluminacion=random.choice(self.fn_data["iluminaciones"]),
                serie=self.fn_data["series"][0],
                et=random.choice(self.fn_data["ET"]),
                pastilla=p,
                ext=self.fn_data["ext"]
            )

            msk_filename = self.fn_data["fn_msk_patt"].format(
                fecha=self.fn_data["fecha"],
                rotacion=r,
                serie=self.fn_data["series"][0],
                pastilla=p,
                ext=self.fn_data["ext"]
            )

            img_path = os.path.join(self.img_dir, img_filename)
            mask_path = os.path.join(self.mask_dir, msk_filename)
            # print(self.samples[i], img_filename, msk_filename) # Avoid excessive printing during training

            img = cv2.imread(img_path, cv2.IMREAD_COLOR)
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

            if img is None:
                print(f"⚠️ Could not load image: {img_path}")
                continue
            if mask is None:
                print(f"⚠️ Could not load mask: {mask_path}")
                continue

            img = cv2.resize(img, (224, 224))
            mask = cv2.resize(mask, (224, 224))

            if self.norm:
                img = img.astype(np.float32) / 255.0
                mask = mask.astype(np.float32) / 255.0
            else:
                img = img.astype(np.float32)
                mask = mask.astype(np.float32)

            # Aumentación: Flip horizontal aleatorio
            if self.augment and random.random() < 0.5:
                img = cv2.flip(img, 1)
                mask = cv2.flip(mask, 1)

            if mask.ndim == 2:
                mask = np.expand_dims(mask, axis=-1)  # ahora shape (224, 224, 1)

            # Ensure image has 3 channels for preprocessing
            if img.ndim == 2:
                img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
            elif img.shape[-1] == 4: # Handle images with alpha channel
                img = cv2.cvtColor(img, cv2.COLOR_BGRA2RGB)

            if img.shape != (224, 224, 3):
                 print(f"⚠️ Unexpected shape for image: {img.shape} ({img_path})")
                 continue

            if mask.shape != (224, 224, 1):
              print(f"⚠️ Unexpected shape for mask: {mask.shape} ({mask_path})")
              continue


            # Preprocesamiento específico de la red (como ResNet152)
            if self.preprocess_fn:
                img = self.preprocess_fn(img)

            batch_imgs.append(img)
            batch_masks.append(mask)

        # Handle case where no valid images/masks were loaded in the batch
        if not batch_imgs:
            return np.empty((0, 224, 224, 3), dtype=np.float32), np.empty((0, 224, 224, 1), dtype=np.float32)


        return np.array(batch_imgs), np.array(batch_masks)

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)

In [ ]:
# X_train, y_train, X_val, y_val, X_test, y_test, filenames_train, filenames_val, filenames_test = augmented_dataset(image_directory, masks_directory, seed=42, norm=False)

In [ ]:
# with open(f'{base_path}/output/filenames_train.txt', 'w', encoding='utf-8') as f:
#     for elemento in filenames_train:
#         f.write(elemento + '\n')
# with open(f'{base_path}/output/filenames_val.txt', 'w', encoding='utf-8') as f:
#     for elemento in filenames_val:
#         f.write(elemento + '\n')
# with open(f'{base_path}/output/filenames_test.txt', 'w', encoding='utf-8') as f:
#     for elemento in filenames_test:
#         f.write(elemento + '\n')


In [ ]:
# with open(f'{base_path}/Datasets/train_images.txt', 'r', encoding='utf-8') as f:
#     filenames_train = [linea.strip() for linea in f]

# with open(f'{base_path}/Datasets/test_images.txt', 'r', encoding='utf-8') as f:
#     filenames_test = [linea.strip() for linea in f]

# with open(f'{base_path}/Datasets/val_images.txt', 'r', encoding='utf-8') as f:
#     filenames_val = [linea.strip() for linea in f]

# with open(f'{base_path}/Datasets/train_masks.txt', 'r', encoding='utf-8') as f:
#     filenames_train_masks = [linea.strip() for linea in f]

# with open(f'{base_path}/Datasets/test_masks.txt', 'r', encoding='utf-8') as f:
#     filenames_test_masks = [linea.strip() for linea in f]

# with open(f'{base_path}/Datasets/val_masks.txt', 'r', encoding='utf-8') as f:
#     filenames_val_masks = [linea.strip() for linea in f]

# print(len(filenames_train), len(filenames_test), len(filenames_val))
# print(len(filenames_train_masks), len(filenames_test_masks), len(filenames_val_masks))

In [ ]:
with open(f'{base_path}/Datasets/train_samples.txt', 'r', encoding='utf-8') as f:
    filenames_train = [linea.strip() for linea in f]

with open(f'{base_path}/Datasets/test_samples.txt', 'r', encoding='utf-8') as f:
    filenames_test = [linea.strip() for linea in f]

with open(f'{base_path}/Datasets/val_samples.txt', 'r', encoding='utf-8') as f:
    filenames_val = [linea.strip() for linea in f]

In [ ]:
fn_data = {
  "fecha": "250127",
  "series": ["BA"],
  # "rotaciones": [
  #   "1", "2", "3", "4", "5", "6", "7", "8", "9", "10",
  #   "11", "12", "13", "14", "15", "16", "17", "18", "19", "20",
  #   "21", "22", "23", "24", "25", "26", "27", "28", "29", "30",
  #   "31", "32", "33", "34", "35", "36", "37", "38", "39", "40",
  #   "41", "42", "43", "44", "45", "46", "47", "48", "49", "50"
  # ],
  "iluminaciones": ["0-11000000", "0-00110000", "0-00001100", "0-00000011", "1-11000000"],
  "ET": ["30000.00", "32000.00", "34000.00", "36000.00", "38000.00"],
  # "hileras": ["A", "B", "C", "D", "E", "F"],
  # "filas": ["0", "1", "2", "3", "4"],
  "ext": "png",
  "fn_img_patt": "{fecha}-r{rotacion}-i{iluminacion}-{serie}-ET{et}-a0000_{pastilla}.{ext}",
  "fn_msk_patt": "{fecha}-r{rotacion}-{serie}-avg_{pastilla}.{ext}"
}


train_gen = SegmentationDataGenerator(
    image_directory, masks_directory,
    filenames_train,
    fn_data,
    # filenames_train, filenames_train_masks,
    batch_size=8,
    preprocess_fn=preprocess_input,
    norm=True,
    shuffle=True,
    augment=True
)

val_gen = SegmentationDataGenerator(
    image_directory, masks_directory,
    filenames_val, # filenames_val_masks,
    fn_data,
    batch_size=8,
    preprocess_fn=preprocess_input,
    norm=True,
    shuffle=False,
    augment=False
)

In [ ]:
# train_gen.samples[train_gen.indices[0]]
# r = train_gen[0]

In [ ]:
# # X_train, y_train, X_val, y_val, X_test, y_test = augmented_dataset_ETIl(image_directory, masks_directory,
# X_train, y_train, X_val, y_val, = augmented_dataset_ETIl(image_directory, masks_directory,
#                                                                         filenames_train, filenames_val,
#                                                                         filenames_train_masks, filenames_val_masks,
#                                                                         norm=False)

# 5 - Entrenamiento

##Parámetros

In [ ]:
batch_size=16
epochs=500
patience=30 #20


## Callbacks

In [ ]:
from datetime import datetime

# import datetime
import pytz
from tensorflow.keras.callbacks import Callback
tz = pytz.timezone('America/Argentina/Buenos_Aires')

class EpochTimingCallback(Callback):
    def __init__(self, timezone_str='America/Argentina/Buenos_Aires'):
        super().__init__()
        self.tz = pytz.timezone(timezone_str)

    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start = datetime.now(self.tz)
        print(f"🟢 Epoch {epoch+1} - Inicio: {self.epoch_start.strftime('%Y-%m-%d %H:%M:%S')} (GMT-3)")

    def on_epoch_end(self, epoch, logs=None):
        epoch_end = datetime.now(self.tz)
        duration = epoch_end - self.epoch_start
        print(f"\n🔴 Epoch {epoch+1} - Fin: {epoch_end.strftime('%Y-%m-%d %H:%M:%S')} (GMT-3) - Duración: {duration}")

early_stopping_cb = keras.callbacks.EarlyStopping(patience=patience,monitor='val_loss', restore_best_weights=True) # EarlyStopping(patience=10, restore_best_weights=True),

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, Callback

# === 3. Callback mejorado para guardar historial .pkl y actualizar .csv ===
class HistorySaver(Callback):
    def __init__(self, pkl_path, csv_path):
        super().__init__()
        self.pkl_path = pkl_path
        self.csv_path = csv_path

        # Cargar historial CSV existente si hay
        if os.path.exists(csv_path):
            self.df_history = pd.read_csv(csv_path, index_col="epoch")
        else:
            self.df_history = pd.DataFrame()

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        new_row = pd.DataFrame([logs], index=[epoch])
        new_row.index.name = 'epoch'

        # Agregar nueva fila al historial existente
        self.df_history = pd.concat([self.df_history, new_row])
        self.df_history = self.df_history[~self.df_history.index.duplicated(keep='last')]
        self.df_history.sort_index(inplace=True)

        # Guardar como CSV
        self.df_history.to_csv(self.csv_path)
        print(f"📝 CSV actualizado hasta época {epoch + 1}")

        # También guardar como .pkl para reanudar
        with open(self.pkl_path, 'wb') as f:
            pickle.dump(self.df_history.to_dict(orient='list'), f)

# === 4. Función para verificar historial válido ===
def valid_history(hist):
    return (
        isinstance(hist, dict) and
        'loss' in hist and
        isinstance(hist['loss'], list) and
        len(hist['loss']) > 0
    )

In [ ]:
from datetime import datetime

model_type = "resnet152"
model_id = "20250721_195907_resnet152_pores-m250520"
# model_id = None

if not model_id:
  # Nuevo modelo Identifica la corrida en particular
  now = datetime.now().strftime("%Y%m%d_%H%M%S")
  model_id = f'{now}_{model_type}_{defect}_m250520'
  print(f"Nuevo modelo: {model_id}")
else:
  print(f"Modelo existente: {model_id}")

# Modelo en proceso


Modelo existente: 20250721_195907_resnet152_pores-m250520


##ResNet152

In [ ]:
import pandas as pd
import pickle

# === 2. Rutas ===
out_folder = f'{base_path}/output/models'

checkpoint_path = f"{base_path}/model/checkpoints_{model_id}/last.keras"
history_pkl_path = f"{base_path}/model/checkpoints_{model_id}/history.pkl"
history_csv_path = f"{base_path}/model/checkpoints_{model_id}/history.csv"

# Crear carpeta de logs con timestamp
log_dir = f'{base_path}/output/logs/fit/{model_id}'

os.makedirs(os.path.dirname(checkpoint_path), exist_ok=True)
os.makedirs(out_folder, exist_ok=True)
os.makedirs(log_dir, exist_ok=True)

# Callbacks
timing_cb = EpochTimingCallback()

tensorboard_cb = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
print(f"{model_id}")

# train_gen.preprocess_fn = get_preprocessing('resnet152')
# val_gen.preprocess_fn = get_preprocessing('resnet152')

steps_per_epoch = len(train_gen)
validation_steps = len(val_gen)

if model_type == "resnet152":
  # X_train_resnet = preprocess_input_resnet152(X_train)
  # X_val_resnet = preprocess_input_resnet152(X_val)

  # === 5. Cargar o crear modelo ===
  if os.path.exists(checkpoint_path):
      print(f"🔁 Cargando modelo completo desde: {checkpoint_path}")
      model = tf.keras.models.load_model(
          checkpoint_path,
          custom_objects={
              'custom_bce_dice_loss': custom_bce_dice_loss,
              'custom_bce_jaccard_loss': custom_bce_jaccard_loss, # Add the custom jaccard loss
              'custom_iou_score': custom_iou_score,
              'custom_f1_score': custom_f1_score
          }
      )
  else:
      print("🆕 Creando modelo desde cero.")
      create_model(model_type)
      model = model_resnet_152

  # === 6. Cargar historial si existe ===
  if os.path.exists(history_pkl_path):
      with open(history_pkl_path, 'rb') as f:
          previous_history = pickle.load(f)
      if valid_history(previous_history):
          print(f"📈 Historial cargado con {len(previous_history['loss'])} épocas.")
      else:
          previous_history = None
          os.remove(history_pkl_path)
          print("⚠️ Historial inválido. Se iniciará desde cero.")
  else:
      previous_history = None
      print("📉 No se encontró historial previo.")

  initial_epoch = len(previous_history['loss']) if previous_history else 0

  # === 7. Entrenamiento ===
  print(f'Comienzo entrenamiento: {datetime.now(tz)}')
  history=model.fit(
      # x=X_train_resnet,
      # y=y_train,

      train_gen,
      validation_data=val_gen,

      steps_per_epoch=steps_per_epoch,
      validation_steps=validation_steps,

      # batch_size=batch_size,
      epochs=epochs,
      initial_epoch=initial_epoch,
      # validation_data=(X_val_resnet, y_val),
      callbacks=[
          early_stopping_cb, tensorboard_cb, timing_cb,
          ModelCheckpoint(checkpoint_path, save_best_only=False),
          HistorySaver(history_pkl_path, history_csv_path)
      ],
  )
  print(f'Fin entrenamiento: {datetime.now(tz)}')

  print(f"Saving Model {os.path.join(out_folder,model_id)}.keras...")
  model.save(os.path.join(out_folder,f'{model_id}.keras'))

  print(
      f"batch_size: {batch_size}"," - ",
      f"epochs: {epochs}",
  )

20250721_195907_resnet152_pores-m250520
🔁 Cargando modelo completo desde: /content/drive/MyDrive/Laburo/CNEA/DCA/Proyectos/IV - Inspección Visual/IV/model/checkpoints_20250721_195907_resnet152_pores-m250520/last.keras
📈 Historial cargado con 38 épocas.
Comienzo entrenamiento: 2025-07-22 11:51:51.274189-03:00


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


⚠️ Could not load image: /content/drive/MyDrive/Laburo/CNEA/DCA/Proyectos/IV - Inspección Visual/IV/Images/Output_imágenes y máscaras 20250318/250127_p_2_i11/250127-r13-i0-11000000-BA-ET32000.00-a0000_A1.png
🟢 Epoch 39 - Inicio: 2025-07-22 11:55:04 (GMT-3)
Epoch 39/500
132/132 ━━━━━━━━━━━━━━━━━━━━ 0s 13s/step - custom_f1_score: 0.3912 - custom_iou_score: 0.2658 - loss: 0.7504 
🔴 Epoch 39 - Fin: 2025-07-22 12:33:20 (GMT-3) - Duración: 0:38:16.288460
📝 CSV actualizado hasta época 39
132/132 ━━━━━━━━━━━━━━━━━━━━ 2304s 16s/step - custom_f1_score: 0.3912 - custom_iou_score: 0.2659 - loss: 0.7504 - val_custom_f1_score: 0.1700 - val_custom_iou_score: 0.1002 - val_loss: 0.9202
🟢 Epoch 40 - Inicio: 2025-07-22 12:33:27 (GMT-3)
Epoch 40/500
132/132 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - custom_f1_score: 0.4458 - custom_iou_score: 0.3077 - loss: 0.7048
🔴 Epoch 40 - Fin: 2025-07-22 12:52:35 (GMT-3) - Duración: 0:19:07.442742
📝 CSV actualizado hasta época 40
132/132 ━━━━━━━━━━━━━━━━━━━━ 1155s 9s/step - 

###Evaluación

In [ ]:
%load_ext tensorboard
%tensorboard --logdir "/content/drive/MyDrive/CNEA/DCA/Proyectos/IV - Inspección Visual/IV/output/logs/fit/20250526_184302-model_resnet_152-pores-m250520"

In [ ]:
if model_type == "resnet152":
  #in "real life" it should not be used the test set for the model selection.
  #is just to compare with this toy example
  X_test_resnet = preprocess_input_resnet152(X_test)

  print("Train: ", type(X_train_resnet))
  print("Val: ", type(X_val_resnet))
  print("Test: ", type(X_test_resnet))

  ev_train = model_resnet_152.evaluate(X_train_resnet, y_train)
  ev_val = model_resnet_152.evaluate(X_val_resnet, y_val)
  ev_test = model_resnet_152.evaluate(X_test_resnet, y_test)

  print(f"Train/Val/Test Loss, IoU, F1")
  print(ev_train, '\n', ev_val, '\n', ev_test)


##VGG19

In [ ]:
if model_vgg19 is not None:
  print("VGG19")
  print("Training...")
  # Preprocess input for VGG19

  X_train_vgg = preprocess_input_vgg19(X_train)
  X_val_vgg = preprocess_input_vgg19(X_val)
  X_test_vgg = preprocess_input_vgg19(X_test)

  model_vgg19.fit(
      x=X_train_vgg,
      y=y_train,
      batch_size=batch_size,
      epochs=epochs,
      validation_data=(X_val_vgg, y_val),
      callbacks=[early_stopping_cb]
  )

  print("Saving Model...")
  model_vgg19.save('model_vgg19.keras')

  X_test_vgg = preprocess_input_vgg19(X_test)

  print("vgg 19")
  print("Validation results...")
  model_vgg19.evaluate(X_val_vgg, y_val)

  print("Test results...")
  model_vgg19.evaluate(X_test_vgg, y_test)

  pred_vgg19 = model_vgg19.predict(X_test_vgg)


###Efficientnetb2

In [ ]:
if model_efficientnetb2 is not None:
  print("Efficientnetb2")
  print("Training...")

  X_train_eficient = preprocess_input_efficientnetb2(X_train)
  X_val_eficient = preprocess_input_efficientnetb2(X_val)

  model_efficientnetb2.fit(
      x=X_train_eficient,
      y=y_train,
      batch_size=batch_size,
      epochs=epochs,
      validation_data=(X_val_eficient, y_val),
      callbacks=[early_stopping_cb]
  )

  print("Saving Model...")
  model_efficientnetb2.save('model_efficientnetb2.keras')

  X_test_eficient = preprocess_input_efficientnetb2(X_test)

  print("Efficientnetb2")
  print("Validation results...")
  model_efficientnetb2.evaluate(X_val_eficient, y_val)

  print("Test results...")
  model_efficientnetb2.evaluate(X_test_eficient, y_test)

  pred_efficientnetb2 = model_efficientnetb2.predict(X_test_eficient)


#Test

##Librerias

In [ ]:
import cv2

# def create_mask(pred_mask):
#     thresholded_arr = np.where(pred_mask > 0.5, 1, 0)
#     return thresholded_arr

def aplicar_mascara(imagen, mascara, color = [0, 255, 0]):
    # Convertir la máscara a un arreglo booleano
    mascara = mascara.astype(bool)
    mascara_exp = np.squeeze(mascara)
    # Crear una copia de la imagen original
    imagen_con_mascara = np.copy(imagen)

    # Asignar el valor (0, 255, 0) a los píxeles donde la máscara es verdadera
    imagen_con_mascara[mascara_exp] = color

    return imagen_con_mascara

def save_image(image_array, mask, fname):
    # print(f"save: {fname}")
    # image_array_scaled = (image_array * 255).astype(np.uint8)
    image_array_scaled = (image_array).astype(np.uint8)
    image_mask = aplicar_mascara(image_array_scaled, mask)
    image_out = np.concatenate((image_array_scaled, image_mask), axis=1)
    cv2.imwrite(fname, image_out)

def save_eval(images, predictions, masks, dst_fldr):
    os.makedirs(dst_fldr, exist_ok=True)
    if not len(images) == len(predictions):
        raise(f"El conjunto de imágenes y de predicciones debe tener el mismo tamaño {len(images)} != {len(predictions)}")

    os.makedirs(dst_fldr, exist_ok=True)
    for pos in range(len(predictions)):
        pred = np.where(predictions[pos] <= 0.5, 0, 1)
        save_image(images[pos], pred, os.path.join(dst_fldr,f"{pos:04}.jpg"))
        if masks is not None:
            save_image(images[pos], masks[pos], os.path.join(dst_fldr,f"{pos:04}_mask.jpg"))

def plot_all(images, masks, predictions):
  for i in range(len(predictions)):
    plot_img_mask(np.int64(images[i]), masks[i], predictions[i])

## Entorno

In [ ]:
# model_id = "20250416_165203-model_resnet_152"
# model_id = "20250522_173338-model_resnet_152-pores-m250520"

model_id = "20250421_204907-VGG19-pores"
model_fn = f'{base_path}/output/models/{model_id}.keras'
# 20250421_213428-model_resnet_152-cachaduras'

ds_base_path = f'{base_path}/Images'
# ds_base_path = f'E:/Data/IV/exp20250127'

out_folder = f"{base_path}/output/test/{model_id}"

th = 0.5

##Carga Modelo

In [ ]:
if not os.path.exists(model_fn):
  print(f'Weights file inexistente "{model_fn}"')
else:
  print(f'Cargando modelo "{model_fn}"')
  # model_resnet_152.load_weights(model_fn)
  model_vgg19.load_weights(model_fn)

##Data Test

### Prueba imagenes archivos
Imagenes no necesariamente del dataset de entrenamiento

In [ ]:
from fnmatch import fnmatch

image_directory = f'{ds_base_path}/Output_imágenes y máscaras 20250318/pellets'
# image_directory = f'{ds_base_path}/250127_p_2'
# image_directory = f'{ds_base_path}/Output Img y Mask 20250318/pellets'
# pattern = "250127-r1-i0-*_B4.png"

# image_filenames = [os.path.basename(image_path) for image_path in os.listdir(image_directory) if fnmatch(image_path, pattern)]

# Leer lista desde archivo
with open(f'{base_path}/output/filenames_test.txt', 'r', encoding='utf-8') as f:
    image_filenames = [linea.strip() for linea in f]

images = load_images(image_directory, image_filenames, norm=False)

print(len(images))

In [ ]:
images_resnet = preprocess_input_resnet152(np.array(images))
pred_resnet_152 = model_resnet_152.predict(images_resnet)

In [ ]:
images_vgg19 = preprocess_input_vgg19(np.array(images))
pred_vgg19 = model_vgg19.predict(images_vgg19)

In [ ]:
# predictions = pred_resnet_152
predictions = pred_vgg19
fnames = image_filenames
os.makedirs(out_folder, exist_ok=True)





if not len(images) == len(predictions):
    raise(f"El conjunto de imágenes y de predicciones debe tener el mismo tamaño {len(images)} != {len(predictions)}")

pred_bin = [np.where(p <= th, 0, 1).astype(np.uint8) for p in predictions]

for i in range(len(predictions)):
    image = images[i].astype(np.uint8)
    image_mask = aplicar_mascara(image, pred_bin[i])
    image_out = np.concatenate((image, image_mask), axis=1)

    fname = os.path.join(out_folder, f"{os.path.splitext(fnames[i])[0]}_comp{os.path.splitext(fnames[i])[1]}")
    cv2.imwrite(fname, image_out)

    fname = os.path.join(out_folder, f"{os.path.splitext(fnames[i])[0]}_pred{os.path.splitext(fnames[i])[1]}")
    cv2.imwrite(fname, pred_bin[i] * 255)

###Prueba Test dataset
Conjunto Test del entrenamiento

In [ ]:
# Ejecutar primero la celda de carga de conjunto de entrenamiento
X_test, y_test, filenames_test,  = test_dataset(image_directory, masks_directory, seed=42, norm=False) # Tienen que usar los mismos parámetros que augmented_dataset en el entrenamiento

X_test_resnet = preprocess_input_resnet152(X_test)


In [ ]:
print(type(image_directory), type(masks_directory))

In [ ]:
for i in range(len(pred_resnet_152)):
  plot_img_mask(X_test, y_test, pred_resnet_152)

##Test

In [ ]:
print(type(X_test_resnet), type(y_test))
print(len(X_test_resnet), y_test.shape)

In [ ]:
out_folder = f"{base_path}/output/test/20250416_165203-model_resnet_152"

# os.makedirs(out_folder, exist_ok=True)

X_test_resnet = preprocess_input_resnet152(np.array(X_test))
pred_resnet = model_resnet_152.predict(X_test_resnet)

# loss, iou, f1 = model_resnet_152.evaluate(X_test_resnet, y_test)
# print(f"Test Loss: {loss}, IoU: {iou}, F1: {f1}")



In [ ]:
save_eval(X_test,pred_resnet,y_test,out_folder)

In [ ]:
for i in range(len(y_train)):
  if np.max(y_train[i]) > 0.0 and np.max(y_train[i]) < 255:
    print(i, np.max(y_train[i]), filenames_train[i])

for i in range(len(y_test)):
  if np.max(y_test[i]) > 0.0 and np.max(y_test[i]) < 255:
    print(i, np.max(y_test[i]), filenames_test[i])

for i in range(len(y_val)):
  if np.max(y_val[i]) > 0.0 and np.max(y_val[i]) < 255:
    print(i, np.max(y_val[i]), filenames_val[i])

# np.max(y_train[617])

In [ ]:
print(filenames_test)

In [ ]:
ds_base_path = f'{base_path}/Images'
# ds_base_path = f'E:/Data/IV/exp20250127'
from fnmatch import fnmatch


image_directory = f'{ds_base_path}/Output_imágenes y máscaras 20250318/pellets_B4'
# masks_directory = f'{ds_base_path}/Output_imágenes y máscaras 20250318/all_pores'

# image_directory = f'{ds_base_path}/250127_p_2'
# image_directory = f'{ds_base_path}/Output Img y Mask 20250318/pellets'
# masks_directory = f'{ds_base_path}/Output Img y Mask 20250318/mask_pores'
# print(os.listdir(masks_directory))
pattern = "250127-r1-*_B4.png"
# pattern = "250127-r1-BA-avg_A4.png"

image_filenames = [os.path.basename(image_path) for image_path in os.listdir(image_directory) if fnmatch(image_path, pattern)]
# masks = load_masks(masks_directory, image_filenames)
masks = None
images = load_images(image_directory, image_filenames, norm=False)

print(len(images))
if not masks is None:
    print(len(masks))

In [ ]:
X_test_resnet_short = preprocess_input_resnet152(np.array(images))
pred_resnet_152 = model_resnet_152.predict(X_test_resnet_short)

In [ ]:
out_folder = f"{base_path}/output/test/20250416_165203-model_resnet_152"
os.makedirs(out_folder, exist_ok=True)
print(out_folder)

In [ ]:
predictions = pred_resnet_152
dst_fldr = out_folder
fnames = image_filenames

if not len(images) == len(predictions):
    raise(f"El conjunto de imágenes y de predicciones debe tener el mismo tamaño {len(images)} != {len(predictions)}")

for i in range(len(predictions)):
    pred = np.where(predictions[i] <= 0.5, 0, 1)
    image = images[i].astype(np.uint8)
    image_mask = aplicar_mascara(image, pred.astype(np.uint8))
    image_out = np.concatenate((image, image_mask), axis=1)
    fname = os.path.join(dst_fldr,
                        f"{os.path.splitext(fnames[i])[0]}_pred{os.path.splitext(fnames[i])[1]}"
                         )
    cv2.imwrite(fname, image_out)